In [1]:
import zipfile
import cv2
import numpy as np
from matplotlib import pyplot

from tensorflow.python.client import device_lib

print("Show System RAM Memory:\n\n")
!cat /proc/meminfo | egrep "MemTotal*"
print("\n\nShow Devices:\n\n"+str(device_lib.list_local_devices()))
cv2.__version__

/home/isaiah/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/isaiah/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/isaiah/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/isaiah/anaconda3/envs/tf/lib/python3.6/site-packages

Show System RAM Memory:


MemTotal:        8041524 kB


Show Devices:

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3542228368263938222
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 141819904
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13645532332902043770
physical_device_desc: "device: 0, name: GeForce MX150, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


'4.1.1'

In [2]:
from numpy import asarray

def plot_images(images, n):
    for i in range(n * n):
        if(i >= len(images)):
            break
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data # change from opencv-bgr to matplot-rgb
        pyplot.imshow(images[i][...,::-1].astype('uint8'))
    pyplot.show()

# load all images in a directory into memory
def load_images(start_point, number_limit, archive_name, size=(256,256)):
    # print(dirlist)
    src_list = list()
    archive = zipfile.ZipFile(archive_name, 'r')
    dirlist = archive.namelist()[1:]
    with archive as zfile:

        # enumerate filenames in directory, assume all are images
        for index in range(start_point, start_point + number_limit):
            # load and resize the image

            data = zfile.read(dirlist[index])
            imgfile = cv2.imdecode(np.frombuffer(data, np.uint8), 1)

            resize = cv2.resize(imgfile, size)

            # cv2.imshow("0", resize)
            # cv2.waitKey(10)
            # pixels = img_to_array(resize)
            # print(resize.shape)
            # print(index)
            src_list.append(resize)

    return asarray(src_list)

In [3]:
# load and prepare training images
def prepare_real_samples(file):
    # load compressed arrays
    # data = load(filename)
    # load the face dataset
    # color_images = data['arr_0'].astype('uint8')
    color_images = file
    
    # print(type(src_images))
    gray_images = color_images.astype('uint8')
    gray_images [:,:,:,0] = color_images[:,:,:,0]/3 + color_images[:,:,:,1]/3 + color_images[:,:,:,2]/3
    gray_images [:,:,:,1] = gray_images [:,:,:,0]
    gray_images [:,:,:,2] = gray_images [:,:,:,0]
    
    # unpack arrays
    X1 = gray_images
    X2 = color_images

    plot_images(color_images, 10)
    plot_images(gray_images, 10)
    
    # scale from [0,255] to [-1,1]
    X1 = (X1 - 127.5) / 127.5
    X2 = (X2 - 127.5) / 127.5
    return [X1, X2]



In [6]:
data_path = "/home/isaiah/Downloads/train2014.zip"
model_save_path = "/home/isaiah/GAN_practice_no_git/models/"
summery_path = "/home/isaiah/GAN_practice_no_git/performances/"

archive = zipfile.ZipFile(data_path, 'r')
dirlist = archive.namelist()[1:]

full_size = len(dirlist)
full_size = 40000
part_size = 100

start_step = 0

print(full_size)

load_steps = int(full_size/part_size)
load_steps = 1
load_leftover = full_size%part_size


print("load_steps", load_steps)
print("load_leftover", load_leftover)

#### define everything ####



#### train part!! ####
for step in range(start_step, load_steps + 1):
    start_point = step * part_size

    if (step < load_steps):
        number_limit = part_size
    else:
        number_limit = load_leftover
        if load_leftover == 0:
            break

    src_images = load_images(start_point, number_limit, data_path)
    dataset = prepare_real_samples(src_images)
    print('Loaded', dataset[0].shape, dataset[1].shape)

    #### train here!! ####

    del src_images
    del dataset

FileNotFoundError: [Errno 2] No such file or directory: '/home/isaiah/Downloads/train2014.zip'